In questo notebook possiamo trovare una piccola prova di text-mining realizzata attraverso l'uso di Monkey learn.
Quello che abbiamo fatto è stato prendere le recensioni e applicare un modello di Sentiment Analysis.
Quindi per ogni serie tv abbiamo aggiunto la colonna tag che era la risposta del classificatore(Positivo-neutro-negativo) e una confidenza della risposta data dal classificatore.

In [40]:
import pandas as pd
import re
import numpy as np
from pandas_profiling import ProfileReport
from monkeylearn import MonkeyLearn
import json
from pprint import pprint

In [139]:
pd.options.mode.chained_assignment = None

Quindi come possiamo vedere data una stringa [a], il classificatore ci dava come risposta una file json, con le informazioni di risposta (tag_name) e la sua confidenza di risposta, quindi dovevamo estrapolare queste informazioni da questo file json.

In [102]:
ml = MonkeyLearn('98cea86ee31661c4bbc110b49e867af08fad722e')
data = [a]
model_id = 'cl_pi3C7JiL'
result = ml.classifiers.classify(model_id, data)
 
print(result.body)

[{'text': 'The ostensible subjects of “Age of Samurai: Battle for Japan” are dubbed “the mounted knights of old Japan” and while a properly romantic description, it falls a bit short: The sword-wielding martial artists of the feudal era (1185-1868) have not only taken up residence in the Western imagination but created ornate, interlacing connections between pop cultures East and West. Akira Kurosawa, for instance, who was a student of John Ford, portrayed his heroes like the gunslingers of the Old West, or even Prohibition gangsters. Detective novelist Dashiell Hammett’s 1929 “Red Harvest” was the basis for Kurosawa’s “Yojimbo,” which in turn inspired Sergio Leone’s “A Fistful of Dollars,” which inspired the Bruce Willis movie “Last Man Standing.” Beneath all the intersecting fictional mayhem lay feudal roots. Most of us samurai-movie fans just don’t know where or what they are.The experts in “Age of Samurai” certainly do. And they tell far more than the average viewer is going to be 

Abbiamo realizzato un dataframe prova, in cui abbiamo aggiunto come colonne le chiavi del file json.

In [184]:
prova = pd.DataFrame()
for i in range(0,len(serie)):
    data = []
    data = [serie['Critica'][i]]
    model_id = 'cl_pi3C7JiL'
    result = ml.classifiers.classify(model_id, data)
    prova = prova.append(result.body)
    

Possiamo vedere la nostra colonna Classifications

In [185]:
prova

,text,external_id,error,classifications
0,The ostensible subjects of “Age of Samurai: Ba...,None,False,"[{'tag_name': 'Positive', 'tag_id': 122921383,..."
0,Section: Reception (1):\n\nSubsections (4):\nS...,None,False,"[{'tag_name': 'Positive', 'tag_id': 122921383,..."
0,Section: Reception (1):\n\nSubsections (2):\nS...,None,False,"[{'tag_name': 'Negative', 'tag_id': 122921385,..."
0,Section: Reception (1):\nThe weekend after the...,None,False,"[{'tag_name': 'Negative', 'tag_id': 122921385,..."
0,Section: Reception (1):\nThe review aggregator...,None,False,"[{'tag_name': 'Positive', 'tag_id': 122921383,..."
...,...,...,...,...
0,Section: Reception (1):\nOn the review aggrega...,None,False,"[{'tag_name': 'Negative', 'tag_id': 122921385,..."
0,"The Spanish language show, created by prolific...",None,False,"[{'tag_name': 'Positive', 'tag_id': 122921383,..."
0,t seems like we’re getting a fair amount of st...,None,False,"[{'tag_name': 'Negative', 'tag_id': 122921385,..."
0,Have you ever heard of England described as a ...,None,False,"[{'tag_name': 'Negative', 'tag_id': 122921385,..."


Abbiamo realizzato il Dataframe ciao dal nostro dataframe appena creato prova.
Quindi abbiamo modificato la colonna Classifications rendendola di tipo stringa.

In [188]:
ciao['classifications'] = ciao['classifications'].astype(str)

Poi abbiamo realizzato una RE, in modo da estrapolare solamente l'informazione della Confidence all'interno delle righe della colonna classifications.

In [190]:
def conf(x):
    re_comp1 = re.compile('(\d?\.\d+)')
    m = re_comp1.search(x)
    
    if m:
        return float(m.group(1))

Abbiamo realizzato una RE che estraesse la tag di risposta del classificatore.

In [191]:
def metti(x):
    re_comp1 = re.compile("(Negative)")
    re_comp2 = re.compile('(Positive)')
    m = re_comp1.search(x)
    m_a = re_comp2.search(x)
    
    if m:
        return str(m.group(1))
    if m_a:
        return str(m_a.group(1))
    

Abbiamo applicato le funzioni appena create

In [192]:
ciao['tag_name'] = ciao['classifications'].apply(metti)

In [193]:
ciao['confidence'] = ciao['classifications'].apply(conf)

Ed infine abbiamo creato il nostro csv finale contenente le informazioni di cui necessitavamo

In [195]:
ciao.to_csv('./text_mining_serietv.csv')

In [206]:
text_mining_serietv = pd.read_csv('./text_mining_serietv.csv')
text_mining_serietv

,text,external_id,error,classifications,tag_name,confidence
0,The ostensible subjects of “Age of Samurai: Ba...,NaN,False,"[{'tag_name': 'Positive', 'tag_id': 122921383,...",Positive,0.705
1,Section: Reception (1):\n\nSubsections (4):\nS...,NaN,False,"[{'tag_name': 'Positive', 'tag_id': 122921383,...",Positive,0.531
2,Section: Reception (1):\n\nSubsections (2):\nS...,NaN,False,"[{'tag_name': 'Negative', 'tag_id': 122921385,...",Negative,0.941
3,Section: Reception (1):\nThe weekend after the...,NaN,False,"[{'tag_name': 'Negative', 'tag_id': 122921385,...",Negative,0.587
4,Section: Reception (1):\nThe review aggregator...,NaN,False,"[{'tag_name': 'Positive', 'tag_id': 122921383,...",Positive,0.547
...,...,...,...,...,...,...
63,Section: Reception (1):\nOn the review aggrega...,NaN,False,"[{'tag_name': 'Negative', 'tag_id': 122921385,...",Negative,0.666
64,"The Spanish language show, created by prolific...",NaN,False,"[{'tag_name': 'Positive', 'tag_id': 122921383,...",Positive,0.934
65,t seems like we’re getting a fair amount of st...,NaN,False,"[{'tag_name': 'Negative', 'tag_id': 122921385,...",Negative,0.515
66,Have you ever heard of England described as a ...,NaN,False,"[{'tag_name': 'Negative', 'tag_id': 122921385,...",Negative,0.701


Qui abbiamo semplicemente, sapendo che le righe del dataframe text_mining_serietv facevano riferimento alle stesse righe del dataframe, aggiunto le colonne tag_name e confidence al DF serie

In [208]:
serie['tag_name'] = text_mining_serietv['tag_name']

In [209]:
serie['confidence'] = text_mining_serietv['confidence']

In [211]:
serie.to_csv('./serie_tv_final.csv')

In [210]:
serie

,Titolo,Paese,Produzione,Distribuzione,Cast,Creatore,Durata_episodio,Genere,N_stagioni,N_episodi,Sommario,Cast_testo,Produzione_testo,Critica,Average_rating,Num_votes,tag_name,confidence
0,Age of Samurai: Battle for Japan,United States,Netflix,Netflix,Masayoshi Haneda Masami Kosaka Hideaki Itō Hay...,NaN,44,"Documentary,History,War",1,6,Age of Samurai: Battle for Japan is an America...,NaN,NaN,The ostensible subjects of “Age of Samurai: Ba...,7.5,2218,Positive,0.705
1,Agents of S.H.I.E.L.D.,United States,ABC Studios Marvel Television Mutant Enemy Pro...,Walt Disney Television,Clark Gregg Ming-Na Wen Brett Dalton Chloe Ben...,Joss Whedon Jed Whedon Maurissa Tancharoen,42,Action Drama Science fiction Superhero,7,136,Marvel's Agents of S.H.I.E.L.D. is an American...,Section: Cast and characters (1):\nClark Gregg...,Section: Production (1):\n\nSubsections (8):\n...,Section: Reception (1):\n\nSubsections (4):\nS...,7.5,201304,Positive,0.531
2,Alice in Borderland,Japan,Robot Communications Inc.,Netflix,Kento Yamazaki Tao Tsuchiya Yūki Morinaga Keit...,"Antoine Rodelet, Josselin Charier",46,Science fiction[1] Suspense-thriller[2] Drama[3],1,8,"Alice in Borderland (Japanese: 今際の国のアリス, Hepbu...",Section: Cast and characters (1):\n\nSubsectio...,Section: Production (1):\n\nSubsections (3):\n...,Section: Reception (1):\n\nSubsections (2):\nS...,7.7,18897,Negative,0.941
3,Alien Worlds,United Kingdom,Wall to Wall Media,Netflix,NaN,NaN,44,Docufiction Science fiction Speculative evolution,1,4,Alien Worlds is a British sci-fi nature docufi...,NaN,NaN,Section: Reception (1):\nThe weekend after the...,6.5,2683,Negative,0.587
4,Behind Her Eyes,United Kingdom,Sony Pictures Television Left Bank Pictures,Netflix,Eve Hewson Tom Bateman Simona Brown Robert Ara...,Steve Lightfoot,50,"Psychological thriller,Supernatural fiction",1,6,Behind Her Eyes is a British supernatural psyc...,Section: Cast and characters (1):\nEve Hewson ...,Section: Production (1):\n\nSubsections (3):\n...,Section: Reception (1):\nThe review aggregator...,7.2,32779,Positive,0.547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Virgin River,United States,Reel World Management,Netflix,Alexandra Breckenridge Martin Henderson Colin ...,NaN,44,Romantic drama,2,20,Virgin River is an American romantic drama str...,NaN,Section: Production (1):\n\nSubsections (3):\n...,Section: Reception (1):\nOn the review aggrega...,7.5,21155,Negative,0.666
64,Who Killed Sara?,Mexico,Perro Azul,Netflix,NaN,José Ignacio Valenzuela,40,Thriller Mystery Melodrama,1,1,Who Killed Sara? (Spanish: ¿Quién mató a Sara?...,NaN,NaN,"The Spanish language show, created by prolific...",6.7,1064,Positive,0.934
65,You vs. Wild,USA,Bear Grylls Ventures,Netflix,Bear Grylls,Robert BuchtaBear GryllsDelbert Shoopman,19,"\tAdventure,reality television",1,8,You vs. Wild is a 2019 interactive American ad...,NaN,NaN,t seems like we’re getting a fair amount of st...,6.7,2204,Negative,0.515
66,Zero Chill,United Kingdom,Lime Pictures,Netflix,Grace Beedie Dakota Benjamin Taylor Jade Ma Je...,Kirstie Falkous John Regier,30,Teen drama[1],1,1,Zero Chill is a British teen drama series that...,Section: Cast and characters (1):\n\nSubsectio...,Section: Production (1):\nProducers of Zero Ch...,Have you ever heard of England described as a ...,5.6,630,Negative,0.701
